In [1]:
pip install salesforce-lavis

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 35.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device set to:", device)

Device set to: cuda


In [4]:
from lavis.models import load_model_and_preprocess
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip_feature_extractor", model_type="base", is_eval=True, device=device)

2024-08-10 18:30:16.993821: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 18:30:16.993928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 18:30:17.117228: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 1.97G/1.97G [00:57<00:00, 36.9MB/s] 


In [5]:
import pandas as pd
df = pd.read_csv('/kaggle/input/7th-aug-images/updated_dataset.csv')

In [6]:
df['path'] = df['path'].str.replace('7th_aug_images','7th-aug-images')

In [7]:
df.head()

,Unnamed: 0,text,image_url,label,path
0,0,craigslist ad get paid to attend harvard for me,https://external-preview.redd.it/jt6tIxka4W2mj...,0,/kaggle/input/7th-aug-images/images/image_0.jpg
1,1,this cat chillin on an arm rest,https://preview.redd.it/ygvtnvtoqb831.jpg?widt...,0,/kaggle/input/7th-aug-images/images/image_1.jpg
2,2,Disney Edits Controversial Mandalorian Scene S...,https://media.babylonbee.com/articles/article-...,1,/kaggle/input/7th-aug-images/images/image_2.jpg
3,3,Hero: This Man Refused To Turn To His Neighbor...,https://media.babylonbee.com/articles/article-...,1,/kaggle/input/7th-aug-images/images/image_3.jpg
4,4,bentley says national leaders should look to a...,https://external-preview.redd.it/cpqJ9WmQhmFRA...,0,/kaggle/input/7th-aug-images/images/image_4.jpg


In [8]:
len(df)

21885

In [ ]:
from PIL import Image
import pandas as pd

multimodal_embeddings = []
image_embeddings = []
text_embeddings = []

for index, row in df.iterrows():
    image_path = row['path']
    image = Image.open(image_path).convert("RGB")
    
    text = row['text']
    text_input = txt_processors["eval"](text)
    
    image_processed = vis_processors["eval"](image).unsqueeze(0).to(device)
    sample = {"image": image_processed, "text_input": [text_input]}
    
    multimodal_emb = model.extract_features(sample).multimodal_embeds[0,0,:] 
    image_emb = model.extract_features(sample, mode="image").image_embeds[0,0,:] 
    text_emb = model.extract_features(sample, mode="text").text_embeds[0,0,:]
    
    multimodal_embeddings.append(multimodal_emb.cpu().numpy())
    image_embeddings.append(image_emb.cpu().numpy())
    text_embeddings.append(text_emb.cpu().numpy())

df['Multimodal Embeddings'] = multimodal_embeddings
df['Image Embeddings'] = image_embeddings
df['Text Embeddings'] = text_embeddings


In [ ]:
import numpy as np

In [ ]:
multimodal_embeddings =np.array(multimodal_embeddings)
text_embeddings = np.array(text_embeddings)
image_embeddings = np.array(image_embeddings)

In [ ]:
multimodal_inputs = multimodal_embeddings  
image_inputs = image_embeddings       
text_inputs = text_embeddings        
labels = np.array(df['label'])             

In [ ]:
from sklearn.model_selection import train_test_split

train_multimodal, test_multimodal, train_image, test_image, train_text, test_text, train_labels, test_labels = train_test_split(
    multimodal_inputs, image_inputs, text_inputs, labels, test_size=0.2, random_state=42, stratify=labels)

val_multimodal, test_multimodal, val_image, test_image, val_text, test_text, val_labels, test_labels = train_test_split(
    test_multimodal, test_image, test_text, test_labels, test_size=0.5, random_state=42, stratify=test_labels)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder



X_multimodal_train, X_multimodal_val, X_image_train, X_image_val = train_multimodal, val_multimodal,train_image,val_image 
y_train=torch.tensor(train_labels)
y_val=torch.tensor(val_labels)

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel1(nn.Module):
    def __init__(self, multimodal_dim, image_dim, output_dim):
        super(MyModel1, self).__init__()
        self.fc1 = nn.Linear(multimodal_dim + image_dim, 1024)  
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)
        self.fc7 = nn.Linear(32, 16)
        self.fc8 = nn.Linear(16, 8)
        self.fc9 = nn.Linear(8, 4)  
        self.fc10 = nn.Linear(4, output_dim)  
        self.relu = nn.ReLU()
        self.output_dim = output_dim
    
    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.relu(self.fc6(x))
        x = self.relu(self.fc7(x))
        x = self.relu(self.fc8(x))
        x = self.relu(self.fc9(x))  
        x = self.fc10(x)
        return x




output_dim = 2
model = MyModel1(multimodal_dim=multimodal_embeddings.shape[1], 
                image_dim=image_embeddings.shape[1], 
                output_dim=output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200
best_loss = float('inf')
early_stopping_counter = 0
patience = 5 
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(torch.tensor(X_multimodal_train, dtype=torch.float), 
                    torch.tensor(X_image_train, dtype=torch.float))
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
    
    model.eval()
    with torch.no_grad():
        outputs = model(torch.tensor(X_multimodal_val, dtype=torch.float), 
                        torch.tensor(X_image_val, dtype=torch.float))
        val_loss = criterion(outputs, y_val)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_val, predicted)
        print(f'Validation Loss: {val_loss}')
        
        if val_loss < best_loss:
            best_loss = val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("Early stopping!")
                break


In [ ]:
torch.save(model.state_dict(), 'model_im.pth')


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_text_train, X_text_val= train_text, test_text

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel2(nn.Module):
    def __init__(self, multimodal_dim, text_dim, output_dim):
        super(MyModel2, self).__init__()
        self.fc1 = nn.Linear(multimodal_dim + text_dim, 1024)  
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)
        self.fc7 = nn.Linear(32, 16)
        self.fc8 = nn.Linear(16, 8)
        self.fc9 = nn.Linear(8, 4)  
        self.fc10 = nn.Linear(4, output_dim)  
        self.relu = nn.ReLU()
        self.output_dim = output_dim
    
    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.relu(self.fc6(x))
        x = self.relu(self.fc7(x))
        x = self.relu(self.fc8(x))
        x = self.relu(self.fc9(x))  
        x = self.fc10(x)
        return x

# After train_test_split, check shapes
print("Train Multimodal Shape:", X_multimodal_train.shape)
print("Train Text Shape:", X_text_train.shape)
print("Val Multimodal Shape:", X_multimodal_val.shape)
print("Val Text Shape:", X_text_val.shape)

# Verify that the sizes match before concatenation
if X_multimodal_val.shape[0] != X_text_val.shape[0]:
    print(f"Shape mismatch: Multimodal val {X_multimodal_val.shape[0]} vs Text val {X_text_val.shape[0]}")
    # Optionally, slice the tensors to match sizes if the difference is small
    min_size = min(X_multimodal_val.shape[0], X_text_val.shape[0])
    X_multimodal_val = X_multimodal_val[:min_size]
    X_text_val = X_text_val[:min_size]
    y_val = y_val[:min_size]

# Concatenate and continue with training
X_val = torch.cat((torch.tensor(X_multimodal_val, dtype=torch.float), 
                   torch.tensor(X_text_val, dtype=torch.float)), dim=1)

output_dim = 2
model = MyModel2(multimodal_dim=multimodal_embeddings.shape[1], 
                text_dim=text_embeddings.shape[1], 
                output_dim=output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200
best_loss = float('inf')
early_stopping_counter = 0
patience = 5  
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(torch.tensor(X_multimodal_train, dtype=torch.float), 
                    torch.tensor(X_text_train, dtype=torch.float))
    loss = criterion(outputs, torch.tensor(y_train, dtype=torch.long))
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
    
    model.eval()
    with torch.no_grad():
        outputs = model(torch.tensor(X_multimodal_val, dtype=torch.float), 
                        torch.tensor(X_text_val, dtype=torch.float))
        val_loss = criterion(outputs, torch.tensor(y_val, dtype=torch.long))
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_val, predicted)
        print(f'Validation Loss: {val_loss}')
        
        if val_loss < best_loss:
            best_loss = val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("Early stopping!")
                break


In [ ]:
torch.save(model.state_dict(), 'model_tm.pth')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel3(nn.Module):
    def __init__(self, image_dim, text_dim, output_dim):
        super(MyModel3, self).__init__()
        self.fc1 = nn.Linear(image_dim + text_dim, 1024)  
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)
        self.fc7 = nn.Linear(32, 16)
        self.fc8 = nn.Linear(16, 8)
        self.fc9 = nn.Linear(8, 4)  
        self.fc10 = nn.Linear(4, output_dim)  
        self.relu = nn.ReLU()
        self.output_dim = output_dim
    
    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.relu(self.fc6(x))
        x = self.relu(self.fc7(x))
        x = self.relu(self.fc8(x))
        x = self.relu(self.fc9(x))  
        x = self.fc10(x)
        return x




output_dim = 2
model = MyModel3(image_dim=image_embeddings.shape[1], 
                text_dim=text_embeddings.shape[1], 
                output_dim=output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200
best_loss = float('inf')
early_stopping_counter = 0
patience = 5  
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(torch.tensor(X_image_train, dtype=torch.float), 
                    torch.tensor(X_text_train, dtype=torch.float))
    loss = criterion(outputs, torch.tensor(y_train, dtype=torch.long))
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
    
    model.eval()
    with torch.no_grad():
        outputs = model(torch.tensor(X_image_val, dtype=torch.float), 
                        torch.tensor(X_text_val, dtype=torch.float))
        val_loss = criterion(outputs, torch.tensor(y_val, dtype=torch.long))
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_val, predicted)
        print(f'Validation Loss: {val_loss}')
        
        if val_loss < best_loss:
            best_loss = val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("Early stopping!")
                break


In [ ]:
torch.save(model.state_dict(), 'model_it.pth')


In [ ]:
model1 = MyModel1(multimodal_dim=multimodal_embeddings.shape[1], 
                image_dim=image_embeddings.shape[1], 
                output_dim=output_dim)

In [ ]:
model3 = MyModel3(text_dim=text_embeddings.shape[1], 
                image_dim=image_embeddings.shape[1], 
                output_dim=output_dim)

In [ ]:
model2 = MyModel2(multimodal_dim=multimodal_embeddings.shape[1], 
                text_dim=text_embeddings.shape[1], 
                output_dim=output_dim)

In [ ]:
model1.load_state_dict(torch.load('/kaggle/working/model_im.pth'))
model2.load_state_dict(torch.load('/kaggle/working/model_it.pth'))
model3.load_state_dict(torch.load('/kaggle/working/model_tm.pth'))


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FusionModel(nn.Module):
    def __init__(self, model1, model2, model3, output_dim):
        super(FusionModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        
        self.fc1 = nn.Linear(in_features=model1.output_dim + model2.output_dim + model3.output_dim, out_features=128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, output_dim) 
        self.relu = nn.ReLU()
    
    def forward(self, multimodal_input, image_input, text_input):
        multimodal_output = self.model1(multimodal_input, image_input)
        image_output = self.model3(image_input, text_input)
        text_output = self.model2(multimodal_input, text_input)
        fused_output = torch.cat((multimodal_output, image_output, text_output), dim=1)
        fused_output = self.relu(self.fc1(fused_output))
        fused_output = self.relu(self.fc2(fused_output))
        fused_output = self.relu(self.fc3(fused_output))
        final_output = self.fc4(fused_output)
        return final_output


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, multimodal_inputs, image_inputs, text_inputs, labels):
        self.multimodal_inputs = multimodal_inputs
        self.image_inputs = image_inputs
        self.text_inputs = text_inputs
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        multimodal_input = torch.tensor(self.multimodal_inputs[index], dtype=torch.float)
        image_input = torch.tensor(self.image_inputs[index], dtype=torch.float)
        text_input = torch.tensor(self.text_inputs[index], dtype=torch.float)
        label = torch.tensor(self.labels[index], dtype=torch.long)
        return multimodal_input, image_input, text_input, label

train_dataset = CustomDataset(train_multimodal, train_image, train_text, train_labels)
val_dataset = CustomDataset(val_multimodal, val_image, val_text, val_labels)
test_dataset = CustomDataset(test_multimodal, test_image, test_text, test_labels)

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
def train_model(model, train_dataloader, val_dataloader, criterion, optimizer, device, num_epochs=10, patience=5):
    model.to(device)
    model.train()
    best_val_loss = float('inf')
    early_stopping_counter = 0
    f1_score = F1Score(task="binary", num_classes=2, average='macro').to(device)  # Assuming binary classification

    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        for multimodal_input, image_input, text_input, labels in train_dataloader:
            multimodal_input = multimodal_input.to(device)
            image_input = image_input.to(device)
            text_input = text_input.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(multimodal_input, image_input, text_input)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        epoch_loss = running_loss / total_samples
        epoch_accuracy = correct_predictions / total_samples

        model.eval()
        val_correct_predictions = 0
        val_total_samples = 0
        val_running_loss = 0.0
        with torch.no_grad():
            for multimodal_input, image_input, text_input, labels in val_dataloader:
                multimodal_input = multimodal_input.to(device)
                image_input = image_input.to(device)
                text_input = text_input.to(device)
                labels = labels.to(device)
                outputs = model(multimodal_input, image_input, text_input)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * labels.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct_predictions += (predicted == labels).sum().item()
                val_total_samples += labels.size(0)
        val_loss = val_running_loss / val_total_samples
        val_accuracy = val_correct_predictions / val_total_samples
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("Early stopping!")
                break
            
    print('Training finished.')


In [ ]:
import torch.optim as optim

fusion_model = FusionModel(model1, model2, model3, output_dim=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fusion_model.parameters(), lr=0.001)

In [ ]:
import torch
from torchmetrics import F1Score

def test_model(model, test_dataloader, device):
    model.eval()
    correct_predictions = 0
    total_samples = 0
    f1_score = F1Score(task="binary", num_classes=2, average='macro').to(device)

    with torch.no_grad():
        for multimodal_input, image_input, text_input, labels in test_dataloader:
            multimodal_input = multimodal_input.to(device)
            image_input = image_input.to(device)
            text_input = text_input.to(device)
            labels = labels.to(device)

            outputs = model(multimodal_input, image_input, text_input)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            f1_score.update(predicted, labels)  # Update the F1Score metric

    test_accuracy = correct_predictions / total_samples
    test_f1_score = f1_score.compute()  # Compute the F1Score for the test set

    print(f'Test Accuracy: {test_accuracy:.4f}, Test F1-Score: {test_f1_score:.4f}')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_model(fusion_model, train_dataloader, val_dataloader, criterion, optimizer, device, num_epochs=50)



In [ ]:
test_model(fusion_model, test_dataloader, device)